In [ ]:
%pwd
%cd C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price

C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price


In [1]:
import praw
import pandas as pd
import time
from datetime import datetime, timedelta
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --- CONFIGURATION ---

# List of time filters to check, ordered from newest to oldest
# Note: These are only used for the initial large historical fetch.
TIME_FILTERS = ["day", "week", "month", "year"]

# Define the start date for historical fetch (as before)
start_date_str = "2024-06-15 00:00:00"
start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

# Polling intervals in seconds
LIVE_STREAM_INTERVAL = 60    # Stream new posts/comments every 3 seconds
HISTORICAL_FETCH_INTERVAL = 14400 # Fetch historical data every 2 days (300 seconds)

# CSV file path
CSV_FILE = "reddit_crypto_sentiment_analysis.csv"

# --- INITIALIZATION ---

# Initialize Reddit API
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Define cryptocurrency keywords to track
crypto_keywords = {
    "bitcoin": "BTC", "btc": "BTC",
    "ethereum": "ETH", "eth": "ETH",
    "cardano": "ADA", "ada": "ADA",
    "dogecoin": "DOGE", "doge": "DOGE",
    "solana": "SOL", "sol": "SOL"
}

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize a set to prevent processing the same post/comment multiple times
processed_ids = set()

# Initialize timers
last_historical_fetch_time = time.time() - HISTORICAL_FETCH_INTERVAL - 1 # Ensures it runs immediately
last_live_stream_time = time.time() - LIVE_STREAM_INTERVAL - 1 # Ensures it runs immediately

# Initialize CSV file (only once)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)

def truncate_to_interval(dt, interval_hours):
    """
     Truncates a datetime object down to the start of the nearest
    interval_hours block (e.g., 00:00, 04:00, 08:00 for 4 hours).
    This ensures that price and sentiment timestamps align perfectly for joins.
    """
    # Seconds since midnight
    total_seconds = (dt - dt.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()

    # Calculate the number of seconds in the interval
    interval_seconds = interval_hours * 3600

    # Calculate the start of the current interval (integer division)
    truncated_seconds = (total_seconds // interval_seconds) * interval_seconds

    # Reconstruct the datetime object by adding the truncated seconds to midnight
    return dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=truncated_seconds)




print("🔄 Starting Reddit post & comment sentiment streaming...")

# ----------------------------------------------------------------------
# HELPER FUNCTION FOR DATA PROCESSING
# ----------------------------------------------------------------------

def process_submission(post, post_type):
    """Processes a single PRAW submission (post) and its comments, extracts sentiment, and adds to records."""
    
    # Check 1: Has this post been processed already? (Prevents duplicates)
    if post.id in processed_ids:
        return [] # Return empty if already processed

    post_timestamp = post.created_utc
    
    # Check 2: Ignore posts/comments older than the global start date for the main loop
    # This check is primarily for the historical fetch, but good for consistency.
    if post_timestamp < start_timestamp:
        return []

    records = []
    
    try:
        # --- Post Processing ---
        title = getattr(post, 'title', '')
        review = getattr(post, 'selftext', '') if hasattr(post, 'selftext') else ''
        user_id = str(post.author) if post.author else "Anonymous"
        #date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")
        
        
        # --- Inside process_submission function ---

        dt_utc = datetime.utcfromtimestamp(post_timestamp)
        # Truncate to the nearest 4-hour interval
        truncated_dt = truncate_to_interval(dt_utc, interval_hours=4)
        date = truncated_dt.strftime("%Y-%m-%d %H:%M:%S")

        # Detect which crypto is mentioned
        text_lower = (title + " " + review).lower()
        found_coin = None
        for keyword, symbol in crypto_keywords.items():
            if keyword in text_lower:
                found_coin = symbol
                break
        
        if not found_coin:
            return [] # Skip if no tracked crypto is mentioned

        # Sentiment for post
        sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
        records.append([date, user_id, "post", title, found_coin, review, sentiment])
        processed_ids.add(post.id)
        
        # --- Comment Processing ---
        if post_type == "historical":
            # For historical, only load a few comments to save time/requests
            limit = 10
            post.comments.replace_more(limit=0)
            comments = post.comments.list()[:limit]
        else: # For live stream, load more aggressively
            limit = 25
            post.comments.replace_more(limit=1)
            comments = post.comments.list()[:limit]

        for comment in comments:
            if isinstance(comment, praw.models.Comment):
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")

                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                comment_lower = comment_text.lower()
                comment_coin = found_coin
                # Re-check for crypto in the comment itself
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break
                
                # Use a combined ID for comment to track it
                comment_id = f"{post.id}_{comment.id}"
                if comment_id in processed_ids:
                    continue
                
                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])
                processed_ids.add(comment_id)
                
    except Exception as e:
        print(f"Error processing submission/comment {post.id}: {e}")
        return []

    return records

# ----------------------------------------------------------------------
# DATA FETCHERS
# ----------------------------------------------------------------------

def fetch_historical_data():
    """Fetches historical data using the TIME_FILTERS, runs every 5 minutes."""
    
    print(f"\n--- ⏳ Running Historical Fetch (Every {HISTORICAL_FETCH_INTERVAL}s) ---")
    new_records = []
    
    for time_filter in TIME_FILTERS:
        print(f"--- Checking Top posts for: {time_filter.upper()} ---")

        # Fetch the top posts for the current time filter
        for post in reddit.subreddit("CryptoCurrency").top(time_filter=time_filter, limit=None):
            
            # Stop if we hit posts older than the start date on the 'year' filter
            if time_filter == "year" and post.created_utc < start_timestamp:
                print(f"--- Stopping historical fetch: Hit post older than {start_date_str} on 'year' filter. ---")
                return new_records # Return what we have so far

            records = process_submission(post, post_type="historical")
            if records:
                new_records.extend(records)
                print(f"✅ Historical Post ({time_filter}): {datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')} | {records[0][4]} | Sentiment: {records[0][-1]:+.3f}")
    
    return new_records


def fetch_live_stream_data():
    """Fetches the newest data using 'new' or 'stream' approach, runs every 3 seconds."""
    
    print(f"\n--- ⚡️ Running Live Stream Fetch (Every {LIVE_STREAM_INTERVAL}s) ---")
    new_records = []
    
    # Use the 'new' sort for a quick look at the latest posts (limit of 25)
    for post in reddit.subreddit("CryptoCurrency").new(limit=25):
        records = process_submission(post, post_type="live")
        if records:
            new_records.extend(records)
            print(f"🔥 Live Post: {datetime.utcfromtimestamp(post.created_utc).strftime('%H:%M:%S')} | {records[0][4]} | Sentiment: {records[0][-1]:+.3f}")
            
    return new_records

def save_records_to_csv(records):
    """Appends collected records to the CSV file."""
    if records:
        df = pd.DataFrame(records, columns=["date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"])
        df.to_csv(CSV_FILE, mode="a", header=False, index=False)
        print(f"\n💾 SAVED {len(records)} new entries to {CSV_FILE}")

# ----------------------------------------------------------------------
# MAIN LOOP
# ----------------------------------------------------------------------

try:
    while True:
        current_time = time.time()
        new_data_to_save = []

        # 1. Historical Fetch (Every 5 minutes)
        if current_time - last_historical_fetch_time >= HISTORICAL_FETCH_INTERVAL:
            historical_records = fetch_historical_data()
            new_data_to_save.extend(historical_records)
            last_historical_fetch_time = current_time # Reset timer

        # 2. Live Stream Fetch (Every 3 seconds)
        if current_time - last_live_stream_time >= LIVE_STREAM_INTERVAL:
            live_records = fetch_live_stream_data()
            new_data_to_save.extend(live_records)
            last_live_stream_time = current_time # Reset timer
            
        # 3. Save any collected data
        save_records_to_csv(new_data_to_save)

        # Calculate time to sleep until the next live stream fetch
        # This keeps the loop running consistently every 3 seconds
        sleep_time = LIVE_STREAM_INTERVAL - (time.time() - current_time)
        if sleep_time > 0:
            time.sleep(sleep_time)
            
except KeyboardInterrupt:
    print("\n\n🛑 Script interrupted by user (Ctrl+C). Shutting down.")
except Exception as e:
    print(f"\n\n🚨 An unexpected error occurred: {e}")
    
print("Processing complete.")

🔄 Starting Reddit post & comment sentiment streaming...

--- ⏳ Running Historical Fetch (Every 14400s) ---
--- Checking Top posts for: DAY ---


C:\Users\yashs\AppData\Local\Temp\ipykernel_54252\4101367168.py:111: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_utc = datetime.utcfromtimestamp(post_timestamp)
C:\Users\yashs\AppData\Local\Temp\ipykernel_54252\4101367168.py:147: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
C:\Users\yashs\AppData\Local\Temp\ipykernel_54252\4101367168.py:199: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: da

✅ Historical Post (day): 2025-11-17 06:29:21 | BTC | Sentiment: +0.000
✅ Historical Post (day): 2025-11-17 20:28:10 | BTC | Sentiment: +0.000
✅ Historical Post (day): 2025-11-17 06:19:50 | BTC | Sentiment: -0.572
✅ Historical Post (day): 2025-11-17 13:40:21 | BTC | Sentiment: +0.000
✅ Historical Post (day): 2025-11-17 19:12:11 | BTC | Sentiment: -0.998
✅ Historical Post (day): 2025-11-17 03:22:20 | ADA | Sentiment: -0.318
✅ Historical Post (day): 2025-11-17 15:26:48 | BTC | Sentiment: +0.000
✅ Historical Post (day): 2025-11-17 14:23:20 | BTC | Sentiment: -0.951
✅ Historical Post (day): 2025-11-17 13:46:25 | BTC | Sentiment: +0.000
✅ Historical Post (day): 2025-11-17 02:46:00 | BTC | Sentiment: +0.869
✅ Historical Post (day): 2025-11-17 20:22:37 | BTC | Sentiment: +0.831
✅ Historical Post (day): 2025-11-17 01:01:14 | ETH | Sentiment: +0.954
✅ Historical Post (day): 2025-11-17 08:26:47 | BTC | Sentiment: -0.670
✅ Historical Post (day): 2025-11-17 16:24:09 | BTC | Sentiment: +0.000
✅ Hist

In [1]:
%pwd

'C:\\Users\\yashs\\AppData\\Local\\Programs\\Microsoft VS Code'

In [ ]:
%pwd 

%cd C:\Users\yashs\source\repos\Ripa-Shah\Sentiment-Analysis\Analyzing-Social-Sentiment-Data-to-determine-correlation-with-Cryptocurrency-Price

import praw
import pandas as pd
import time
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# --- CONFIGURATION ---

# List of time filters to check, ordered from newest to oldest
TIME_FILTERS = ["day", "week", "month", "year"]

# Define the start date: June 15, 2025
start_date_str = "2025-06-15 00:00:00"
start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

# Polling intervals in seconds
LIVE_STREAM_INTERVAL = 172800   # Stream new posts/comments every 2 days
HISTORICAL_FETCH_INTERVAL = 172800 # Fetch historical data every 2 days 

# CSV file path
CSV_FILE = "reddit_crypto_sentiment.csv"

# --- INITIALIZATION ---

# Initialize Reddit API (NOTE: Ensure these credentials are valid)
reddit = praw.Reddit(
    client_id="M9iSExe3kRkwrILVO75-CA",
    client_secret="rnuYodBu7llgTAmssHujZVUt37mrAA",
    user_agent="crypto-sentiment-app by u/MindlessEssay2919"
)

# Test PRAW Connection
try:
    test_post = next(reddit.subreddit("CryptoCurrency").hot(limit=1))
    print(f"✅ PRAW Connection successful. Found post: {test_post.title}")
except Exception as e:
    print(f"❌ PRAW Connection failed! Error: {e}")
    print("Please check your client_id and client_secret.")
    exit() # Stop the script if the connection fails

# Define cryptocurrency keywords (simplified for efficiency)
crypto_keywords = {
    "bitcoin": "BTC", "btc": "BTC",
    "ethereum": "ETH", "eth": "ETH",
    "cardano": "ADA", "ada": "ADA",
    "dogecoin": "DOGE", "doge": "DOGE",
    "solana": "SOL", "sol": "SOL"
}

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Initialize a set to prevent processing the same post/comment multiple times
processed_ids = set()

# Initialize timers (set to run immediately on first loop)
last_historical_fetch_time = time.time() - HISTORICAL_FETCH_INTERVAL - 1
last_live_stream_time = time.time() - LIVE_STREAM_INTERVAL - 1

# Initialize CSV file (creates the file with headers)
df_init = pd.DataFrame(columns=[
    "date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"
])
df_init.to_csv(CSV_FILE, index=False)
print(f"File {CSV_FILE} initialized successfully.")

print(f"\n🔄 Starting Reddit sentiment stream (Tracking since {start_date_str})...")

# ----------------------------------------------------------------------
# HELPER FUNCTIONS
# ----------------------------------------------------------------------

def process_submission(post, post_type):
    """Processes a single PRAW submission (post) and its comments."""
    
    # Check 1: Has this post been processed already? 
    if post.id in processed_ids:
        return []

    post_timestamp = post.created_utc
    
    # Check 2: Ignore posts older than the starting date
    if post_timestamp < start_timestamp:
        return []

    records = []
    
    try:
        # --- Post Processing ---
        title = getattr(post, 'title', '')
        review = getattr(post, 'selftext', '') if hasattr(post, 'selftext') else ''
        user_id = str(post.author) if post.author else "Anonymous"
        date = datetime.utcfromtimestamp(post_timestamp).strftime("%Y-%m-%d %H:%M:%S")

        # Detect crypto (text is already lowercased for checking efficiency)
        text_lower = (title + " " + review).lower()
        found_coin = None
        for keyword, symbol in crypto_keywords.items():
            if keyword in text_lower:
                found_coin = symbol
                break
        
        if not found_coin:
            return []

        # Sentiment for post
        sentiment = analyzer.polarity_scores(title + " " + review)["compound"]
        records.append([date, user_id, "post", title, found_coin, review, sentiment])
        processed_ids.add(post.id)
        
        # --- Comment Processing ---
        limit = 10 if post_type == "historical" else 25
        
        # This resolves MoreComments objects to actual comments
        post.comments.replace_more(limit=0 if post_type == "historical" else 1)
        comments = post.comments.list()[:limit]

        for comment in comments:
            if isinstance(comment, praw.models.Comment):
                comment_text = comment.body
                comment_user = str(comment.author) if comment.author else "Anonymous"
                
                if not comment_text or comment_text in ["[deleted]", "[removed]"]:
                    continue

                comment_id = f"{post.id}_{comment.id}"
                if comment_id in processed_ids:
                    continue
                
                comment_date = datetime.utcfromtimestamp(comment.created_utc).strftime("%Y-%m-%d %H:%M:%S")
                comment_lower = comment_text.lower()
                comment_coin = found_coin # Inherit post coin initially

                # Re-check for crypto in the comment itself
                for keyword, symbol in crypto_keywords.items():
                    if keyword in comment_lower:
                        comment_coin = symbol
                        break
                
                comment_sent = analyzer.polarity_scores(comment_text)["compound"]
                records.append([comment_date, comment_user, "comment", title, comment_coin, comment_text, comment_sent])
                processed_ids.add(comment_id)
                
    except Exception as e:
        # Catch errors like deleted posts/authors/comments
        # print(f"Error processing submission/comment {post.id}: {e}") 
        return []

    return records


def fetch_historical_data():
    """Fetches historical data using the TIME_FILTERS, runs every 5 minutes."""
    
    print(f"\n--- ⏳ Running Historical Fetch (Top posts since {start_date_str}) ---")
    new_records = []
    
    for time_filter in TIME_FILTERS:
        # Fetch the top posts for the current time filter (PRAW's limit is ~1000)
        for post in reddit.subreddit("CryptoCurrency").top(time_filter=time_filter, limit=None):
            
            # This handles stopping the deep search once we hit very old, irrelevant content
            if time_filter == "year" and post.created_utc < start_timestamp:
                print(f"--- Stopping historical fetch: Hit content older than {start_date_str} ---")
                return new_records 

            records = process_submission(post, post_type="historical")
            if records:
                new_records.extend(records)
                print(f"✅ Historical Post ({time_filter}): {datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')} | {records[0][4]} | Sentiment: {records[0][-1]:+.3f}")
    
    return new_records


def fetch_live_stream_data():
    """Fetches the newest data using 'new' sort, runs every 3 seconds."""
    
    print(f"\n--- ⚡️ Running Live Stream Fetch (Every {LIVE_STREAM_INTERVAL}s) ---")
    new_records = []
    
    # Use the 'new' sort for a quick look at the latest posts (limit of 25)
    for post in reddit.subreddit("CryptoCurrency").new(limit=25):
        records = process_submission(post, post_type="live")
        if records:
            new_records.extend(records)
            print(f"🔥 Live Post: {datetime.utcfromtimestamp(post.created_utc).strftime('%H:%M:%S')} | {records[0][4]} | Sentiment: {records[0][-1]:+.3f}")
            
    return new_records


def save_records_to_csv(records):
    """Appends collected records to the CSV file."""
    if records:
        df = pd.DataFrame(records, columns=["date", "user_id", "type", "title", "cryptocurrency", "review", "sentiment_score"])
        # Append data to the CSV without writing the header again
        df.to_csv(CSV_FILE, mode="a", header=False, index=False)
        print(f"\n💾 SAVED {len(records)} new entries to {CSV_FILE}")

# ----------------------------------------------------------------------
# MAIN LOOP
# ----------------------------------------------------------------------

try:
    while True:
        current_time = time.time()
        new_data_to_save = []

        # 1. Historical Fetch (Every 5 minutes)
        if current_time - last_historical_fetch_time >= HISTORICAL_FETCH_INTERVAL:
            historical_records = fetch_historical_data()
            new_data_to_save.extend(historical_records)
            last_historical_fetch_time = current_time # Reset timer

        # 2. Live Stream Fetch (Every 3 seconds)
        if current_time - last_live_stream_time >= LIVE_STREAM_INTERVAL:
            live_records = fetch_live_stream_data()
            new_data_to_save.extend(live_records)
            last_live_stream_time = current_time # Reset timer
            
        # 3. Save any collected data
        if new_data_to_save:
            save_records_to_csv(new_data_to_save)
        
        # Calculate time to sleep to maintain the 3-second live stream interval
        time_spent = time.time() - current_time
        sleep_time = max(0, LIVE_STREAM_INTERVAL - time_spent)
        
        if sleep_time > 0:
            time.sleep(sleep_time)
            
except KeyboardInterrupt:
    print("\n\n🛑 Script interrupted by user (Ctrl+C). Shutting down.")
except Exception as e:
    print(f"\n\n🚨 An unexpected error occurred: {e}")
    
print("Processing complete.")

In [20]:
pip install yfinance pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime
# --- Configuration ---

start_date_str = "2024-06-15 00:00:00"
HISTORICAL_START_DATE = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

TICKERS = ["BTC-USD", "ETH-USD", "ADA-USD", "DOGE-USD", "SOL-USD"]
#HISTORICAL_START_DATE = "2024-06-15"
HISTORICAL_INTERVAL = "5d"  # 5-minute intervals
CSV_FILE_HISTORICAL = "crypto_prices.csv"

print(f"\n--- Fetching Historical Data from {HISTORICAL_START_DATE} at {HISTORICAL_INTERVAL} intervals ---")

try:
    # Use start= and interval= for historical data
    df_historical_wide = yf.download(
        tickers=TICKERS,
        start=HISTORICAL_START_DATE,
        interval=HISTORICAL_INTERVAL,
        progress=False
    )["Close"]

    # Convert the wide format DataFrame to a long (melted) format
    # This aligns with the structure of your CSV output (timestamp, symbol, price)
    df_historical_wide.index.name = "timestamp"
    
    df_historical_long = df_historical_wide.reset_index().melt(
        id_vars='timestamp',
        value_vars=TICKERS,
        var_name='symbol',
        value_name='price'
    ).dropna() # Remove rows where price data might be missing

    # Save to CSV
    df_historical_long.to_csv(CSV_FILE_HISTORICAL, index=False)

    print(f"\n✅ Historical data successfully retrieved and saved to {CSV_FILE_HISTORICAL}")
    print(f"Total historical records: {len(df_historical_long):,}")

except Exception as e:
    print(f"\n❌ Error retrieving historical data: {e}")

C:\Users\yashs\AppData\Local\Temp\ipykernel_11732\2972712628.py:19: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_historical_wide = yf.download(

5 Failed downloads:
['ADA-USD', 'ETH-USD', 'SOL-USD', 'DOGE-USD', 'BTC-USD']: ValueError("Unable to parse input dt 1718434800.0 of type <class 'float'>")



--- Fetching Historical Data from 1718434800.0 at 5d intervals ---

✅ Historical data successfully retrieved and saved to crypto_prices.csv
Total historical records: 0


In [ ]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime

# --- Configuration ---
TICKERS = ["BTC-USD", "ETH-USD", "ADA-USD", "DOGE-USD", "SOL-USD"]
CSV_FILE_LIVE = "crypto_prices.csv"
LIVE_UPDATE_INTERVAL = "5"  # Seconds

# Initialize CSV file with headers (only once)
df_init = pd.DataFrame(columns=["timestamp", "symbol", "price"])
df_init.to_csv(CSV_FILE_LIVE, index=False)

print(f"Starting live crypto price retrieval every {LIVE_UPDATE_INTERVAL} seconds...")

try:
    while True:
        # Get latest 1-minute bar data. '1m' is the smallest interval yfinance supports.
        # We request the last 5 minutes of data ('5m') to ensure we catch the latest bar.
        data = yf.download(
            tickers=TICKERS,
            period="1d",
            interval="5d",
            progress=False
        )["Close"].tail(1)

        # Check if data was successfully retrieved (Yahoo sometimes returns NaN/empty)
        if not data.empty and not data.iloc[0].isnull().all():
            #datetm = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            start_date_str = datetime.now()
            #start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()

            #start_timestamp = datetime.strptime(start_date_str, "%Y-%m-%d %H:%M:%S").timestamp()
            start_timestamp = datetime.utcfromtimestamp(start_date_str).strftime("%Y-%m-%d %H:%M:%S")

            #timestamp = datetime.now().strptime(datetm,"%Y-%m-%d %H:%M:%S").timestamp()
            records = []

  


            # Iterate through available data (only tickers present in data)
            for symbol in TICKERS:
                try:
                    price = data[symbol].values[0]
                    records.append([start_timestamp, symbol, price])
                    print(f"{start_timestamp} | {symbol} | ${price:.4f}")
                except IndexError:
                    pass

            # Append to CSV file
            df = pd.DataFrame(records, columns=["timestamp", "symbol", "price"])
            df.to_csv(CSV_FILE_LIVE, mode="a", header=False, index=False)

            
        else:
            print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | No new price data retrieved.")

        # Wait for 3 seconds before the next update
        time.sleep(int(LIVE_UPDATE_INTERVAL))

except KeyboardInterrupt:
    print("\nLive streaming stopped by user.")


SyntaxError: invalid syntax (2852824729.py, line 31)

In [51]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime, timedelta # <- timedelta is REQUIRED for the function
import os

# --- Configuration ---
TICKERS = ["BTC-USD", "ETH-USD", "ADA-USD", "DOGE-USD", "SOL-USD"]
HISTORICAL_START_DATE = "2024-06-15"
# NOTE: The Reddit script must use this same interval (4 hours = 4)
HISTORICAL_INTERVAL = "4h" 
CSV_FILE = "crypto_prices.csv"
LIVE_UPDATE_INTERVAL = 5  # Use an integer/float for time.sleep()
LIVE_FETCH_INTERVAL = "1m" 
file_exists = os.path.exists(CSV_FILE)

# ----------------------------------------------------------------------
# HELPER FUNCTION FOR TIME ALIGNMENT (MOVED HERE TO PREVENT NameError)
# ----------------------------------------------------------------------

def truncate_to_interval(dt, interval_hours):
    """
    Truncates a datetime object down to the start of the nearest
    interval_hours block (e.g., 00:00, 04:00, 08:00 for 4 hours).
    This ensures that price and sentiment timestamps align perfectly for joins.
    """
    # Seconds since midnight
    total_seconds = (dt - dt.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
    
    # Calculate the number of seconds in the interval
    interval_seconds = interval_hours * 3600
    
    # Calculate the start of the current interval (integer division)
    truncated_seconds = (total_seconds // interval_seconds) * interval_seconds
    
    # Reconstruct the datetime object by adding the truncated seconds to midnight
    return dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=truncated_seconds)

# ----------------------------------------------------------------------
# 1. Historical Data Retrieval and CSV Initialization
# ----------------------------------------------------------------------
print(f"--- 1. Fetching Historical Data from {HISTORICAL_START_DATE} at {HISTORICAL_INTERVAL} intervals ---")

try:
    # Fetch historical data
    df_historical_wide = yf.download(
        tickers=TICKERS,
        start=HISTORICAL_START_DATE,
        interval=HISTORICAL_INTERVAL,
        progress=False
    )["Close"]

    # Convert to long format
    df_historical_wide.index.name = "timestamp"
    df_historical_long = df_historical_wide.reset_index().melt(
        id_vars='timestamp',
        value_vars=TICKERS,
        var_name='symbol',
        value_name='price'
    ).dropna() # Remove rows where price data might be missing

    # Historical data timestamps are already truncated to the start of the interval 
    # by yfinance. We only need to format them as a string.
    df_historical_long['timestamp'] = df_historical_long['timestamp'].dt.strftime("%Y-%m-%d %H:%M:%S")

    # Save to CSV
    if not file_exists:
        df_historical_long.to_csv(CSV_FILE, index=False, mode='w')
    else:
        df_historical_long.to_csv(CSV_FILE, index=False, mode='a', header=False) # Use header=False when appending

    print(f"\n✅ Historical data successfully retrieved and saved to {CSV_FILE}")
    print(f"Total historical records: {len(df_historical_long):,}")

except Exception as e:
    print(f"\n❌ Error retrieving historical data: {e}")
    # If historical fetch fails, still initialize the file with only the header
    pd.DataFrame(columns=["timestamp", "symbol", "price"]).to_csv(CSV_FILE, index=False, mode='w')


# ----------------------------------------------------------------------
# 2. Live Data Streaming and Appending
# ----------------------------------------------------------------------

print(f"\n--- 2. Starting Live Crypto Price Retrieval every {LIVE_UPDATE_INTERVAL} seconds ---")

try:
    while True:
        # Get the latest data point
        data = yf.download(
            tickers=TICKERS,
            period="1d",
            interval=LIVE_FETCH_INTERVAL,
            progress=False
        )["Close"].tail(1) # Take only the last row

        # Check if data was successfully retrieved
        if not data.empty and not data.iloc[0].isnull().all():
            
            current_dt = datetime.now()
            
            # --- APPLY TIME TRUNCATION HERE ---
            # Truncate the live price timestamp to match the 4-hour bar interval
            truncated_dt = truncate_to_interval(current_dt, interval_hours=4)
            timestamp = truncated_dt.strftime("%Y-%m-%d %H:%M:%S")
            # -----------------------------------
            
            records = []

            # Iterate through available data
            for symbol in TICKERS:
                try:
                    price = data[symbol].values[0] 
                    
                    if not pd.isna(price): # Ensure price is not NaN
                        records.append([timestamp, symbol, price])
                        print(f"{timestamp} | {symbol} | ${price:.4f}")
                        
                except Exception:
                    # Handles cases where a symbol might be missing from the DataFrame
                    pass

            # Append the new live data to the CSV file
            if records:
                df_live = pd.DataFrame(records, columns=["timestamp", "symbol", "price"])
                # Use mode='a' to append, and header=False to avoid writing the header again
                df_live.to_csv(CSV_FILE, mode="a", header=False, index=False)
            else:
                print(f"{timestamp} | No new price data retrieved/data was NaN.")

            # Wait for the configured interval
            time.sleep(LIVE_UPDATE_INTERVAL)
            
except KeyboardInterrupt:
    print("\nLive streaming stopped by user.")
except TypeError as e:
    print(f"\nError: {e}. Check that LIVE_UPDATE_INTERVAL is a number (integer/float).")

--- 1. Fetching Historical Data from 2024-06-15 at 4h intervals ---


C:\Users\yashs\AppData\Local\Temp\ipykernel_11732\1254874932.py:46: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_historical_wide = yf.download(



✅ Historical data successfully retrieved and saved to crypto_prices.csv
Total historical records: 15,620

--- 2. Starting Live Crypto Price Retrieval every 5 seconds ---


C:\Users\yashs\AppData\Local\Temp\ipykernel_11732\1254874932.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


2025-11-17 12:00:00 | ADA-USD | $0.4632


...

C:\Users\yashs\AppData\Local\Temp\ipykernel_11732\1254874932.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


2025-11-17 12:00:00 | ADA-USD | $0.4631


C:\Users\yashs\AppData\Local\Temp\ipykernel_11732\1254874932.py:90: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(


2025-11-17 12:00:00 | ADA-USD | $0.4631


PermissionError: [Errno 13] Permission denied: 'crypto_prices.csv'